In [1]:
# Import the libraries we will need
import pandas as pd
import numpy as np

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import pyspark.pandas as ps

# create the Spark Session
spark = SparkSession.builder.appName("Q1").getOrCreate()

c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [32]:
browsing.take(5)

[Row(Items=['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222', '']),
 Row(Items=['GRO99222', 'GRO12298', 'FRO12685', 'ELE91550', 'SNA11465', 'ELE26917', 'ELE52966', 'FRO90334', 'SNA30755', 'ELE17451', 'FRO84225', 'SNA80192', '']),
 Row(Items=['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO86643', '']),
 Row(Items=['ELE17451', 'ELE37798', 'FRO86643', 'GRO56989', 'ELE23393', 'SNA11465', '']),
 Row(Items=['ELE17451', 'SNA69641', 'FRO86643', 'FRO78087', 'SNA11465', 'GRO39357', 'ELE28573', 'ELE11375', 'DAI54444', ''])]

In [47]:
s = 100 # support threshold

browsing = spark.read.csv("hw1-bundle/hw1-bundle/q2/data/browsing.txt", sep='\t')
browsing = browsing.toDF("Items")
browsing = browsing.withColumn("Items", F.split("Items", " ").cast("array<string>"))

In [49]:
browsing_exploded = browsing.withColumn("Item", F.explode(browsing["Items"]))
browsing_exploded = browsing_exploded.filter(F.length("Item") == 8)
browsing_grouped = browsing_exploded.groupBy("Item").count().withColumnRenamed("count", "Frequency")
freq_items = browsing_grouped.filter(F.col("Frequency") >= s)

In [50]:
browsing_grouped.show(5)

+--------+---------+
|    Item|Frequency|
+--------+---------+
|DAI29878|      114|
|SNA96271|     1295|
|DAI45339|      159|
|ELE38641|        2|
|SNA56589|       18|
+--------+---------+
only showing top 5 rows



In [51]:
freq_items.count()

647

In [63]:
candidate_pairs = freq_items.select("Item").crossJoin(freq_items.select("Item")).toDF("Item_1", "Item_2").filter(F.col("Item_1") < F.col("Item_2"))

In [64]:
candidate_pairs.show(5)

+--------+--------+
|  Item_1|  Item_2|
+--------+--------+
|DAI29878|SNA96271|
|DAI29878|DAI45339|
|DAI29878|ELE67905|
|DAI29878|SNA40078|
|DAI29878|SNA59903|
+--------+--------+
only showing top 5 rows



In [65]:
candidate_pairs = candidate_pairs.crossJoin(browsing)

In [68]:
candidate_pairs = candidate_pairs.withColumn("is_pair_in_Items", ((F.col("Item_1").isin("Items")) & (F.col("Item_2").isin("Items"))).cast("int"))

In [69]:
candidate_pairs.show(10)

+--------+--------+--------------------+----------------+
|  Item_1|  Item_2|               Items|is_pair_in_Items|
+--------+--------+--------------------+----------------+
|DAI29878|SNA96271|[FRO11987, ELE174...|               0|
|DAI29878|SNA96271|[GRO99222, GRO122...|               0|
|DAI29878|SNA96271|[ELE17451, GRO734...|               0|
|DAI29878|SNA96271|[ELE17451, ELE377...|               0|
|DAI29878|SNA96271|[ELE17451, SNA696...|               0|
|DAI29878|SNA96271|[ELE17451, GRO734...|               0|
|DAI29878|SNA96271|[ELE17451, ELE599...|               0|
|DAI29878|SNA96271|[ELE17451, SNA696...|               0|
|DAI29878|SNA96271|[ELE17451, GRO734...|               0|
|DAI29878|SNA96271|[ELE17451, SNA696...|               0|
+--------+--------+--------------------+----------------+
only showing top 10 rows



In [71]:
candidate_pairs = candidate_pairs.groupBy(["Item_1", "Item_2"]).agg(F.sum("is_pair_in_Items").alias("Frequency"))

In [73]:
candidate_pairs.show(5)